# Entropy (validation)

Date : 22/06/2018  <br>
Author : SL   <br>
Code : https://github.com/bosscha/alma  <br>

## Scope

To validate (or not) we perform simulations with C43-X on a trial of sources and we compare the results with N random array configurations taken from the underlying pads. <br>
We show the entropy analysis in this nb. <br>
<br>
Simulations produced by *runValidateEntropy.py*


In [105]:
import os, shutil , glob , pickle
import matplotlib.pyplot as pl
from pylab import rcParams
import numpy as np
import math
from sklearn.metrics import mutual_info_score
from numpy import unique
from scipy.stats import entropy as scipy_entropy


from astropy.io import fits
from astropy.stats import histogram
from astropy.wcs import WCS


%matplotlib inline

RAD2ARCSEC = 180. / math.pi

In [106]:
def MutualInformation(im1, im2, binsize = 20):
    "Compute the mutual information between im1 and im2"
    
    
    hgram, x_edges, y_edges = np.histogram2d(im1.ravel(), im2.ravel(), bins = binsize)
    
    pxy = hgram / float(np.sum(hgram))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0 # Only non-zero pxy values contribute to the sum
    mi = np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))
    
    return(mi)

In [107]:
def isFitsN(n, nforward=1):
    "Check if fits n is in the products directory. rootname s the string basis of the fits"
    
    fitsname = "simRan.compskymodel.flat.regrid.conv.fits.%d"%(n)
    
    if os.path.exists(fitsname):
        return(True)
    else :
        fitsname = "simRan.compskymodel.flat.regrid.conv.fits.%d"%(n+nforward)
        if os.path.exists(fitsname):
            return(True)
        else :
            return(False)
    
    
def load_nant(fitsname):
    "find the NANT for the fitsname"
    
    nant = -1
    fileant = "nantran.txt"
    with open(fileant) as f:
        data = f.read()
        field = data.split()
        
        for i in range(len(field)-1):
            if field[i] == fitsname:
                nant = int(field[i+1])
        
    return(nant)
            
        

In [108]:
def compute_mistats(arrStdName, binsize = 20, resClipping = False, Tol = 0.1):
    "Compute the MI for the trial and arrStdName"
    
    
    miTarget = []
    miRandom = []
    resRandom = []
    nantRandom = []
    
    index = 0
    while(isFitsN(index)):
        sim1   = "%s.image.fits.%d"%(arrStdName,index)
        model1 = "%s.compskymodel.flat.regrid.conv.fits.%d"%(arrStdName,index)   
        with fits.open(model1) as m1:
            in1 = m1[0].data
            bmin1 = m1[0].header['bmin'] * 3600.
            bmaj1 = m1[0].header['bmaj'] * 3600.
            bres1 = math.sqrt(bmin1 * bmaj1)
            
        with fits.open(sim1) as s1:
            out1 = s1[0].data
        mi1  = MutualInformation(in1 , out1 , binsize)  
 
        
        sim2   = "simRan.image.fits.%d"%(index)
        model2 = "simRan.compskymodel.flat.regrid.conv.fits.%d"%(index)
        
        try:
            with fits.open(model2) as m2:
                in2 = m2[0].data
                nant = load_nant(sim2)
                bmin2 = m2[0].header['bmin'] * 3600.
                bmaj2 = m2[0].header['bmaj'] * 3600.
                bres2 = math.sqrt(bmin2 * bmaj2)


            with fits.open(sim2) as s2:
                out2 = s2[0].data
            
            mi2  = MutualInformation(in2 , out2, binsize)
        
            diffRes = abs(bres2-bres1) / bres1
            miTarget.append(mi1)
            if resClipping and diffRes < Tol:
                miRandom.append(mi2)
                resRandom.append(bres2)
                nantRandom.append(nant)
            
            if not resClipping:
                miRandom.append(mi2)
                resRandom.append(bres2)
                nantRandom.append(nant)
        
        except:
            print("## Problem with fits: %s"%(sim2))
        
        index += 1
        
        
    print("## %d images found"%(len(miRandom)))
    return(miTarget, miRandom, resRandom, nantRandom, bres1)    

In [109]:
def pickle_result(filename, data):  
    with open(filename, 'w') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [110]:
def read_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
        
    return(data)

### Analysis ...

The clip boolean allows to select the random array in a resolution range close to the target resolution (within the tolerance Tol)

In [ ]:
# wdir = "/home/stephane/Science/ALMA/ArrayConfig/imaging/fullcombination/simEntropy/products"
# wdir = "/home/stephane/Science/ALMA/ArrayConfig/imaging/entropy/simentropy/products"
# wdir = "/home/stephane/Science/ALMA/ArrayConfig/imaging/fullcombination/entropyntropy/products"
wdir = "/home/stephane/alma/ArrayConfig/imaging/fullcombination/entropy/products"
os.chdir(wdir)

In [ ]:
clip = False
tolerance = 0.1
bin = 10

miTarget , miRandom, resRandom, nantRandom , resTarget = compute_mistats("alma.cycle5.3", resClipping = clip, Tol = tolerance, binsize = bin)

if  clip :
    print("## Clipping in resolution to %3.1f%%"%(tolerance*100.))
else:
    print("## no clipping")
    
tarnp = np.array(miTarget)
print("\n## Target MI:")
print("## Mean: %f"%(tarnp.mean()))
print("## Std: %f"%(tarnp.std()))
print("## RES: %f"%(resTarget))
print()

rannp  = np.array(miRandom)
ranres = np.array(resRandom)

print("## Random MI:")
print("## Mean: %f"%(rannp.mean()))
print("## Std: %f"%(rannp.std()))
print("## RES(mean): %f"%(ranres.mean()))

rcParams['figure.figsize'] = 15, 10.0

mimax  = max(miRandom)
mimax = 0.05
nantmax = 50
pdfmax = 100
tarmean = tarnp.mean()
tarstd = tarnp.std()

pl.subplot(2, 2, 1)
n1, bins1, patches1 = pl.hist(miTarget, 30, density=True, range = (0., mimax), facecolor='g', alpha=0.4, label = "C43-3", hatch ="//")
n2, bins2, patches2 = pl.hist(miRandom, 30, density=True, range = (0., mimax), facecolor='r', alpha=0.6, label = "Random")
pl.xlabel('MI')
pl.ylabel('Probability')
pl.axis([0., mimax, 0, pdfmax])
pl.legend(loc = "upper right")

pl.subplot(2, 2, 2)
pl.plot(nantRandom , miRandom, "r*", label = "Random")
pl.plot([0,nantmax],[tarmean,tarmean], "g-", label = "C43-3")
pl.plot([0,nantmax],[tarmean+tarstd,tarmean+tarstd], "g--")
pl.plot([0,nantmax],[tarmean-tarstd,tarmean-tarstd], "g--")
pl.xlabel('NANT')
pl.ylabel('MI (nats)')
pl.axis([0., nantmax, 0, mimax])
pl.legend(loc = "upper right")

pl.subplot(2, 2, 3)
n2, bins2, patches2 = pl.hist(resRandom, 30, density=True, facecolor='r', alpha=0.5, label = "Random")
pl.xlabel('resolution (arcsec)')
pl.ylabel('Probability')
pl.plot([resTarget,resTarget],[0.,1000], "g-")
pl.axis([1.0, 2.5, 0, 10.0])
pl.legend(loc = "upper right")
pl.show()

## Problem with fits: simRan.image.fits.31
## Problem with fits: simRan.image.fits.38
## Problem with fits: simRan.image.fits.41
## Problem with fits: simRan.image.fits.43
## Problem with fits: simRan.image.fits.70
